In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import re

The below chunk of code extracts the listing links for the newest homes 

In [2]:
counties = "https://www.zillow.com/browse/homes/"
state = 'in'


request_headers = {
    'accept':
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip,deflate,br',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as session:
    url = counties + state + '/'
    response = session.get(url, headers = request_headers)


soup = BeautifulSoup(response.content, 'html.parser')
print(soup)

def newest(href):
    return href and re.compile('/newest-homes').search(href)

def other(href):
    return href and not re.compile('/newest-homes').search(href)

x = []
for a in soup.find_all(href = newest):
    x.append(a.get('href'))

county_links = ["https://www.zillow.com/browse/homes/" + state + '/' + end for end in x]

<!DOCTYPE html>

<html style="font-family: 'Open Sans';font-size: 93.75%">
<head>
<style>
      *, :after, :before {
        box-sizing: border-box;
      }

      h2 {
        display: block;
        font-size: 28px;
        line-height: 1.3;
        font-weight: 600;
        margin: 0px 0px 15px;
      }

      section {
        display: block;
      }

      ul  {
        list-style: none;
      }

      li {
        display: list-item;
      }

      a {
        text-decoration: none;
        color: #006AFF;
        cursor: pointer;
      }

      a:visited {
          color: #7A48D6;
      }

      .bh-breadcrumbs>li:before {
        font-size: 12px;
        color: #A7A6AB;
        content: "\b7";
        display: inline-block;
        margin: 0 5px;
      }

      .bh-breadcrumbs>li:first-child:before{
        content: normal;
      }

      .bh-breadcrumbs>li a:hover {
        color: #006AFF;
      }

      .bh-breadcrumbs>li a:active {
        color: #0D4599;
      }
    </styl

In [3]:
all_addresses = []
home_links = []

for county in county_links:
    with requests.Session() as session:
        url = county
        response = session.get(url, headers = request_headers)
    soup = BeautifulSoup(response.content,'html.parser')
    
    homes = []
    def homedetails(href):
        return href and re.compile("/homedetails").search(href)

    for a in soup.find_all(href=homedetails):
        homes.append(a.get('href'))
        
    for h in homes:
        home_links.append("https://www.zillow.com" + h)

In [4]:
# colnames = ['link','address','list_price','beds','baths','sqft']

# df = pd.DataFrame(columns = colnames)
# df['link'] = home_links
# df

,link,address,list_price,beds,baths,sqft
0,https://www.zillow.com/homedetails/3849-N-Capi...,NaN,NaN,NaN,NaN,NaN
1,https://www.zillow.com/homedetails/1735-Ellen-...,NaN,NaN,NaN,NaN,NaN
2,https://www.zillow.com/homedetails/517-E-Troy-...,NaN,NaN,NaN,NaN,NaN
3,https://www.zillow.com/homedetails/5747-Buskir...,NaN,NaN,NaN,NaN,NaN
4,https://www.zillow.com/homedetails/2775-Del-Pr...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
6899,https://www.zillow.com/homedetails/1418-Chartr...,NaN,NaN,NaN,NaN,NaN
6900,https://www.zillow.com/homedetails/1514-Corydo...,NaN,NaN,NaN,NaN,NaN
6901,https://www.zillow.com/homedetails/7451-Highwa...,NaN,NaN,NaN,NaN,NaN
6902,https://www.zillow.com/homedetails/3016-Master...,NaN,NaN,NaN,NaN,NaN


In [44]:
colnames = ['link','address','list_price','beds','baths','sqft']

df = pd.DataFrame(columns = colnames)

In [56]:
i = 0
for link in home_links:
    
    with requests.Session() as session:
        url = link
        response = session.get(url, headers = request_headers)
    soup = BeautifulSoup(response.content,'html.parser')
    try:
        add_span = soup.find_all(id = 'ds-chip-property-address')[0]
    except IndexError:
        pass
    address = " ".join(add_span.text.split())

    for summary in soup.find_all(class_ = "ds-summary-row"):
        sum_dat = summary.find_all('span')
        price_cont = sum_dat[2].get_text()
        bd_cont = sum_dat[5].get_text()
        ba_cont = sum_dat[9].get_text()
        sq_cont = sum_dat[13].get_text()
    new_row = pd.Series(data = {'link': link, 'address': address, 'list_price': price_cont, 'beds': bd_cont, 'baths': ba_cont, 'sqft': sq_cont}, name = i)
    df = df.append(new_row, ignore_index=False)
    i += 1
print(df.head())
